In [ ]:
from lyricsgenius import Genius
token = 'utWUfzSpqx5D-0h6kdlftC_wWX9s2iPsMP5T5QP0phbGprepCLAjRBZP3cmnhZhB'
genius = Genius(token, sleep_time=2)

In [ ]:
from utils import ArtistSongs, Artists, Charts, Songs, Lyrics
from utils.Charts import Chart, ChartList
Artists.genius = genius
ArtistSongs.genius = genius
Charts.genius = genius
Songs.genius = genius
Lyrics.genius = genius

In [ ]:
Charts.chart_download()

In [ ]:
#if Songs.song_is_loaded() is not True: Songs.song_load()

In [ ]:
#if Artists.artists_is_loaded() is not True: Artists.artists_load()

In [ ]:
charts = Charts.chart_load()
for chart in charts:
    print(f'Name: {chart.artist.name}; ID: {chart.artist._id}')
    artist = ArtistSongs.artist_load_songs(chart.artist)

In [ ]:
import os
import json

dirs = os.listdir('artist_songs')
for dir in dirs:
    with open(f'artist_songs/{dir}') as file:
        data = json.load(file)
        datas = data['songs']
        for song_data in datas:
            id = song_data['id']

            song = Songs.song(id)
            song_json = song.to_json()
            song_json = json.loads(song_json)
            song_language = song_json['language']
            if song_language == 'en':
                lyric = Lyrics.load(id)
                #print(lyric)
                print(f'Added song {song.title} of {song.artist}')
            else:
                print(f'Song: {song.title} has {song_language} language') #Remove?

            